In [1]:
base_url = "https://blinkit.com/prn/product/prid/{}"
product_ids = [
    "499373", "543", "363754", "442668", "331906", "524797", 
    "555856", "110016", "437645", "403491", "396629", "424615", "556","11020"
]

product_urls = []

for pid in product_ids:
    full_url = base_url.format(pid)
    product_urls.append(full_url)

# Optional: print the URLs
for url in product_urls:
    print(url)


https://blinkit.com/prn/product/prid/499373
https://blinkit.com/prn/product/prid/543
https://blinkit.com/prn/product/prid/363754
https://blinkit.com/prn/product/prid/442668
https://blinkit.com/prn/product/prid/331906
https://blinkit.com/prn/product/prid/524797
https://blinkit.com/prn/product/prid/555856
https://blinkit.com/prn/product/prid/110016
https://blinkit.com/prn/product/prid/437645
https://blinkit.com/prn/product/prid/403491
https://blinkit.com/prn/product/prid/396629
https://blinkit.com/prn/product/prid/424615
https://blinkit.com/prn/product/prid/556
https://blinkit.com/prn/product/prid/11020


In [2]:
import os
import time
import csv
import re
import threading
import datetime
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re
import pandas as pd
from datetime import timedelta

# Initialize WebDriver
driver = webdriver.Chrome()
wait = WebDriverWait(driver, 10)
#"499373","543","363754","442668","331906","524797","555856","110016","437645","403491",
#"556","11020"
# Example product URLs and zip codes
product_ids = ["499373","543","363754","442668","331906","524797","555856","110016","437645","403491","396629","424615","556","11020"] 
base_url = "https://blinkit.com/prn/product/prid/{}"
zip_codes = [

    "400050", "400071","400014", "400069", "400705", "400001",
    # "411030", "411005", "411057", "411058", "411006", "410506",
    # "560001", "560038", "560068", "560076", "560100", "560043",
    # "500101", "501102", "500010", "500044", "500013", "500047",
    # "110002", "110003", "110005", "110011", "110006", "110037"
]

# Mapping ZIP codes to cities
city_mapping = {
    "Mumbai": ["400050", "400071", "400014", "400069", "400705", "400001"],
    "Pune": ["411030", "411005", "411057", "411058", "411006", "410506"],
    "Bangalore": ["560001", "560038", "560068", "560076", "560100", "560043"],
    "Hyderabad": ["500101", "501102", "500010", "500044", "500013", "500047"],
    "Delhi": ["110002", "110003", "110005", "110011", "110006", "110037"]
}

# Assigning location IDs to cities
location_mapping = {
    "Pune": 1,
    "Mumbai": 2,
    "Bangalore": 3,
    "Hyderabad": 4,
    "Delhi": 5
}

# Function to get city name and location ID from ZIP code
def get_city_and_location(zip_code):
    for city, zips in city_mapping.items():
        if zip_code in zips:
            return city, location_mapping.get(city, 0)  # Default to 0 if not found
    return "Unknown", 0

# Extract brand from product name
def extract_brand(product_name):
    words = product_name.split()
    return words[0] if words else "Unknown"

# Get current date details
scraping_date = datetime.date.today().strftime("%Y-%m-%d")
year = datetime.date.today().year
week_number = datetime.date.today().isocalendar()[1]
month = datetime.date.today().month
quarter = (month - 1) // 3 + 1

# CSV Path
csv_file_path = os.path.join(os.getcwd(), 'blinkit_details.csv')

# Open CSV file to write the product details
with open(csv_file_path, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(["Crawl_id",
        "Platform_id", "Platform_Name", "Location_id", "Location_name", "Pincode",
        "Brand_name", "Sku_name","Size","Web_pid","Pdp_page_url","Pdp_page_img_url","OSA", "OSA_status", "Price_RP","Price_SP",  
        "Discount", "Created_on", "Year", "Week", "Month", "Quarter"
    ])

    for product_id in product_ids:
        product_url = base_url.format(product_id)
        for zipcode in zip_codes:
            try:
                # Open product page
                driver.get(product_url)
                print(f"Opened product page for {product_url} and zipcode: {zipcode}")

                # Handle any potential popup
                try:
                    overlay = driver.find_element(By.XPATH, "//div[contains(@class, 'absolute') and @tabindex='-1']")
                    driver.execute_script("arguments[0].style.display = 'none';", overlay)
                    print("Popup closed.")
                except:
                    print("No overlay found.")

                # Click location selector
                location_selector = wait.until(EC.element_to_be_clickable((By.XPATH, "//div[contains(@class, 'LocationBar__DownArrow-sc-x8ezho-5')]")))
                driver.execute_script("arguments[0].click();", location_selector)
                print(f"Clicked location selector for zipcode: {zipcode}")

                #Enter the zipcode
                zipcode_input = wait.until(EC.presence_of_element_located((By.XPATH, "//input[@type='text' and @name='select-locality']")))
                zipcode_input.clear()
                zipcode_input.send_keys(zipcode)
                print(f"Entered zipcode: {zipcode}")
                time.sleep(2)

                # Try clicking the first location directly
                try:
                    first_location = WebDriverWait(driver, 3).until(
                        EC.element_to_be_clickable((By.XPATH, "(//div[contains(@class, 'LocationSearchList__LocationDetailContainer-sc-93rfr7-1')])[1]"))
                    )
                    driver.execute_script("arguments[0].click();", first_location)
                    print(f"Selected first location for zipcode: {zipcode} (on first try)")
                except:
                    print("First location not found — fallback to alternate entry method.")

                    # Clear the input field using clear() and then re-enter
                    zipcode_input.clear()
                    print("Cleared the zipcode.")
                    time.sleep(2)
    
                    # Re-enter the same zipcode
                    try:
                        zipcode_input.send_keys(zipcode)
                        print(f"Re-entered zipcode: {zipcode}")
                    except Exception as e:
                        print(f"Error while re-entering zipcode: {e}")
    
    
                    # Backspace the last digit
                    try:
                        zipcode_input = wait.until(EC.presence_of_element_located((By.XPATH, "//input[@type='text' and @name='select-locality']")))
                        zipcode_input.click()
                        zipcode_input.send_keys(Keys.BACKSPACE)
                        print("Backspaced the last digit.")
                        time.sleep(2)
                    except Exception as e:
                        print(f"Error while backspacing: {e}")
                        time.sleep(2)
                    # Select the first available location
                    try:
                        first_location = wait.until(EC.element_to_be_clickable((By.XPATH, "(//div[contains(@class, 'LocationSearchList__LocationDetailContainer-sc-93rfr7-1')])[1]")))
                        driver.execute_script("arguments[0].click();", first_location)
                        print(f"Selected first location for zipcode: {zipcode}")
                    except Exception as e:
                        print(f"No locations found for {zipcode}: {e}")
                        continue

                # Wait for the page to load
                time.sleep(5)
                # Extract Product Name
                try:
                    product_name_element = wait.until(EC.presence_of_element_located((By.XPATH, "//h1[contains(text(),'Cadbury') or contains(@class, 'tw-text')]")))
                    product_name = product_name_element.text.strip()
                except:
                    product_name = "Not Found"
                
                brand = extract_brand(product_name)
                try:
                    price_text = wait.until(EC.presence_of_element_located((By.XPATH, "(//div[contains(text(), '₹')])[1]"))).text.strip()
                    prices = [x.strip().replace('₹', '').replace(',', '') for x in price_text.split("MRP")]
                
                    # Convert selling price to float or return None if not found
                    selling_price = float(prices[0]) if len(prices) > 0 and prices[0].replace('.', '', 1).isdigit() else None
                
                # Extract only the MRP before % or OFF using regex and convert to float
                    if len(prices) > 1:
                        combined_value = prices[1].strip()
                        print(f"Combined Value: {combined_value}")
                
                        # Case: combined format like "13010% OFF"
                        if combined_value.endswith('% OFF') and combined_value[:-7].isdigit():
                            raw_value = combined_value[:-7]  # Remove '% OFF'
                            mrp_price = float(raw_value)  # MRP = all but last 2 digits
                                # Last 2 digits = discount
                        else:
                            # Normal case using regex
                            mrp_price_match = re.search(r'\d+(?=\s*%|\s*OFF|$)', combined_value)
                            mrp_price = float(mrp_price_match.group()) if mrp_price_match else None
                            
                    else:
                        mrp_price = None
                        
                
                except:
                    selling_price = None
                    mrp_price = None
                
                    
                # Calculate discount separately
                try:
                    if selling_price is not None and mrp_price is not None and mrp_price > 0:
                        discount =int(((mrp_price - selling_price) / mrp_price) * 100)
                        discount = round(discount, 2)
                    else:
                        discount = None
                except:
                    discount = None

               
                if selling_price is None or mrp_price is None:
                    OSA = "Out of Stock"
                    OSA_Status = 0
                else:
                    OSA = "In Stock"
                    OSA_Status = 1
#size = driver.find_element(By.XPATH,"//p[contains(@class, 'ProductVariants__VariantUnitText-sc-')]").text
                try:
                    size = driver.find_element(
                        By.XPATH,
                        "//div[p/span[text()='Unit']]/div"
                    ).text
                except:
                    size = "N/A"

                






                # elements = driver.find_elements(By.XPATH, "//div[contains(@class, 'ProductAttribute__ProductAttributesDescription-sc-') and contains(@class, 'jJVAqC')]")
                # for el in elements:
                #     print(el.text)

       
                try:
                    image_url = driver.find_element(By.XPATH, "//div[contains(@class, 'ProductCarousel__ImageContainer-sc-')]//img").get_attribute("src")
                except:
                    image_url = "N/A"

                # Get City and Location ID
                city_name, location_id = get_city_and_location(zipcode)

                # Write to CSV
                writer.writerow([2100,2, "Blinkit", location_id, city_name, zipcode,
                                  brand, product_name, size,product_id,product_url,
                                  image_url,OSA, OSA_Status, mrp_price, selling_price,discount, scraping_date,
                                  year, week_number, month, quarter])

                print(f"Data written for zipcode {zipcode}")

            except Exception as e:
                print(f"Error processing ZIP {zipcode} for {product_url}: {e}")

# Close WebDriver
driver.quit()
print(f"CSV file has been saved at: {csv_file_path}")


df=pd.read_csv(r"C:\Users\SHIVANI\Desktop\Web_scrapping\blinkit_details.csv")
df = df.replace({pd.NA: None, pd.NaT: None, float('nan'): None, 'nan': None})


import pymysql
from datetime import datetime
 
# Replace with your cloud database details
CLOUD_DB_CONFIG = {
    "host": "database-1.cq5s62s644gu.us-east-1.rds.amazonaws.com",  # Example: 'your-db-instance.us-east-1.rds.amazonaws.com'
    "database": "cadbury",
    "user": "admin",
    "password": "JPT_DA12345",
    "port": 3306  # Default MySQL port
}
 
# Connect to MySQL
conn = pymysql.connect(**CLOUD_DB_CONFIG)
cursor = conn.cursor()
 
# Create Master Table (Permanent Storage)
# cursor.execute('''
#     CREATE TABLE IF NOT EXISTS cadbury_master (
#         id SERIAL PRIMARY KEY,
#         Crawl_id INT,  
#         Platform_id INT,
#         Platform_Name VARCHAR(8),
#         Location_id INT,
#         Location_name VARCHAR(9),
#         Pincode INT,
#         Brand_name VARCHAR(7),
#         Sku_name VARCHAR(54),
#         Size VARCHAR(4),
#         Web_pid VARCHAR(16),
#         Pdp_page_url VARCHAR(60),
#         Pdp_page_img_url VARCHAR(255),
#         OSA VARCHAR(12),
#         OSA_status INT,
#         Price_RP INT,
#         Price_SP INT,
#         Discount INT,
#         Created_on TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
#         Year INT,
#         Week INT,
#         Month INT,
#         Quarter INT
#     )
# ''')
 
# Create Persistent Temporary Table (Crawl blinkit)
cursor.execute('''
    CREATE TABLE IF NOT EXISTS crawl_blinkit (
        id SERIAL PRIMARY KEY,
        Crawl_id INT DEFAULT 1700,  
        Platform_id INT,
        Platform_Name VARCHAR(8),
        Location_id INT,
        Location_name VARCHAR(9),
        Pincode INT,
        Brand_name VARCHAR(7),
        Sku_name VARCHAR(54),
        Size VARCHAR(4),
        Web_pid VARCHAR(16),
        Pdp_page_url VARCHAR(60),
        Pdp_page_img_url VARCHAR(255),
        OSA VARCHAR(12),
        OSA_status INT,
        Price_RP INT,
        Price_SP INT,
        Discount INT,
        Created_on TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
        Year INT,
        Week INT,
        Month INT,
        Quarter INT
    )
''')
 
# # Create Job Tracker Table (Stores Last Run Time)
# cursor.execute('''
#     CREATE TABLE IF NOT EXISTS job_tracker (
#         id SERIAL PRIMARY KEY,
#         last_run TIMESTAMP
#     )
# ''')
 
# Insert New Data into Crawl Amazon (Example Data)
 
# Insert New Data into Crawl blinkit (Example DataFrame: df)
for row in df.itertuples(index=False):
    cursor.execute('''
        INSERT INTO crawl_blinkit (
            Crawl_id, Platform_id, Platform_Name, Location_id, Location_name, Pincode, Brand_name,
            Sku_name, Size, Web_pid, Pdp_page_url, Pdp_page_img_url, OSA, OSA_status,
            Price_RP, Price_SP, Discount, Created_on, Year, Week, Month, Quarter
        )
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    ''', (
        row.Crawl_id, row.Platform_id, row.Platform_Name, row.Location_id, row.Location_name,
        row.Pincode, row.Brand_name, row.Sku_name, row.Size, row.Web_pid,
        row.Pdp_page_url, row.Pdp_page_img_url, row.OSA, row.OSA_status,
        row.Price_RP, row.Price_SP, row.Discount, row.Created_on,
        row.Year, row.Week, row.Month, row.Quarter
    ))
conn.commit()
print("✅ New data inserted into crawl_blinkit table.")
 
# # Check Last Execution Time
# cursor.execute("SELECT last_run FROM job_tracker ORDER BY id DESC LIMIT 1")
# last_run = cursor.fetchone()
 
# current_time = datetime.now()
 
# # If no record exists, insert the first run time
# if last_run is None:
#     print("🕒 First time running the script. Setting initial timestamp.")
#     cursor.execute("INSERT INTO job_tracker (last_run) VALUES (%s)", (current_time,))
#     conn.commit()
# else:
#     last_run_time = last_run[0]
#     time_difference = current_time - last_run_time
 
#     # Move Data to Master Table if 12 Hours Passed
#     if time_difference >= timedelta(hours=12):
#         print(f"🔄 Moving data at {current_time} (Last run: {last_run_time})...")
 
#         cursor.execute('''
#             INSERT INTO master_table (crawl_id, platform_id, Platform_Name, Location_id, Location_name, Pincode, Brand_name,
#             Sku_name, Size, Web_pid, Pdp_page_url, image_url, OSA, OSA_Status, Price_RP, Price_SP, Discount, Created_on, Year, Week, Month, Quarter)
#             SELECT crawl_id, platform_id, Platform_Name, Location_id, Location_name, Pincode, Brand_name,
#             Sku_name, Size, Web_pid, Pdp_page_url, image_url, OSA, OSA_Status, Price_RP, Price_SP, Discount, Created_on, Year, Week, Month, Quarter
#             FROM crawl_amazon
#         ''')
 
#         cursor.execute("DELETE FROM crawl_blinkit")  # Clears only moved data, keeping table intact
#         cursor.execute("UPDATE job_tracker SET last_run = %s WHERE id = (SELECT MAX(id) FROM job_tracker)", (current_time,))
#         conn.commit()
 
#         print("✅ Data moved to master_table and timestamp updated.")
#     else:
#         print(f"⏳ Less than 12 hours since last run ({time_difference}). Skipping move.")
 
# Close Connection
conn.close()




ModuleNotFoundError: No module named 'pandas'